### Import the required libraries and modules:

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision
import numpy as np

### Unzip the data:

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)
%cd /content/gdrive/MyDrive/1_All_Computations/BV_OD_EX

Mounted at /content/gdrive
/content/gdrive/MyDrive/1_All_Computations/BV_OD_EX


In [ ]:
!unzip new_data_augment.zip

Archive:  new_data_augment.zip
   creating: new_data_augment/
   creating: new_data_augment/test/
   creating: new_data_augment/test/image/
  inflating: new_data_augment/test/image/IDRiD_55.jpg  
  inflating: new_data_augment/test/image/IDRiD_56.jpg  
  inflating: new_data_augment/test/image/IDRiD_57.jpg  
  inflating: new_data_augment/test/image/IDRiD_58.jpg  
  inflating: new_data_augment/test/image/IDRiD_59.jpg  
  inflating: new_data_augment/test/image/IDRiD_60.jpg  
  inflating: new_data_augment/test/image/IDRiD_61.jpg  
  inflating: new_data_augment/test/image/IDRiD_62.jpg  
  inflating: new_data_augment/test/image/IDRiD_63.jpg  
  inflating: new_data_augment/test/image/IDRiD_64.jpg  
  inflating: new_data_augment/test/image/IDRiD_65.jpg  
  inflating: new_data_augment/test/image/IDRiD_66.jpg  
  inflating: new_data_augment/test/image/IDRiD_67.jpg  
  inflating: new_data_augment/test/image/IDRiD_68.jpg  
  inflating: new_data_augment/test/image/IDRiD_69.jpg  
  inflating: new_dat

### Construct the Model:

In [ ]:
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(3, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="UNET")
    return model

input_shape = (512, 768, 3)
model = build_unet(input_shape)
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 512, 768, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 512, 768, 64  1792        ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 512, 768, 64  256        ['conv2d[0][0]']                 
 alization)                     )                                                              

### Adding the Metrics:

In [ ]:
def iou(y_true, y_pred):
    def f(y_true, y_pred):
        intersection = (y_true * y_pred).sum()
        union = y_true.sum() + y_pred.sum() - intersection
        x = (intersection + 1e-15) / (union + 1e-15)
        x = x.astype(np.float32)
        return x
    return tf.numpy_function(f, [y_true, y_pred], tf.float32)

smooth = 1e-15
def dice_coef(y_true, y_pred):
    y_true = tf.keras.layers.Flatten()(y_true)
    y_pred = tf.keras.layers.Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true * y_pred)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) + smooth)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

### Training the Model:

In [ ]:
W = 768
H = 512

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_data(path):
    x = sorted(glob(os.path.join(path, "image", "*.jpg")))
    y = sorted(glob(os.path.join(path, "mask", "*.jpg")))
    return x, y

def shuffling(x, y):
    x, y = shuffle(x, y, random_state=42)
    return x, y

def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)  ## (512, 512, 3)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    x = x[:,:,0:3] 

    return x

def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 3])
    return x, y

def tf_dataset(X, Y, batch_size=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(4)
    return dataset

In [ ]:
""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

""" Directory to save files """
create_dir("files")

""" Hyperparameters """
batch_size = 2
lr = 1e-4
num_epochs = 30
model_path = os.path.join("files", "model.h5")
csv_path = os.path.join("files", "data.csv")

""" Dataset """
dataset_path = "new_data_augment"
train_path = os.path.join(dataset_path, "train")
valid_path = os.path.join(dataset_path, "test")

train_x, train_y = load_data(train_path)
train_x, train_y = shuffling(train_x, train_y)
valid_x, valid_y = load_data(valid_path)

print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")

train_dataset = tf_dataset(train_x, train_y, batch_size=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch_size=batch_size)

train_steps = len(train_x)//batch_size
valid_setps = len(valid_x)//batch_size

if len(train_x) % batch_size != 0:
    train_steps += 1
if len(valid_x) % batch_size != 0:
    valid_setps += 1

""" Model """
model = build_unet((H, W, 3))
# model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=Adam(lr), metrics=[dice_coef, iou, Recall(), Precision()])
model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=[dice_coef, iou, Recall(), Precision()])
# model.summary()

callbacks = [
    ModelCheckpoint(model_path, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, min_lr=1e-6, verbose=1),
    CSVLogger(csv_path),
    TensorBoard(),
    EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=False)
]

model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=valid_dataset,
    steps_per_epoch=train_steps,
    validation_steps=valid_setps,
    callbacks=callbacks
)

Train: 318 - 318
Valid: 27 - 27
Epoch 1/30
159/159 [==============================] - ETA: 0s - loss: 0.8476 - dice_coef: 0.1524 - iou: 0.0830 - recall: 0.4106 - precision: 0.4032
Epoch 00001: val_loss improved from inf to 0.94785, saving model to files/model.h5
159/159 [==============================] - 404s 2s/step - loss: 0.8476 - dice_coef: 0.1524 - iou: 0.0830 - recall: 0.4106 - precision: 0.4032 - val_loss: 0.9478 - val_dice_coef: 0.0525 - val_iou: 0.0269 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00 - lr: 1.0000e-04
Epoch 2/30
159/159 [==============================] - ETA: 0s - loss: 0.7601 - dice_coef: 0.2399 - iou: 0.1372 - recall: 0.2678 - precision: 0.6949
Epoch 00002: val_loss improved from 0.94785 to 0.91864, saving model to files/model.h5
159/159 [==============================] - 376s 2s/step - loss: 0.7601 - dice_coef: 0.2399 - iou: 0.1372 - recall: 0.2678 - precision: 0.6949 - val_loss: 0.9186 - val_dice_coef: 0.0818 - val_iou: 0.0427 - val_recall: 0.0194 - val

### Evaluate The Metrics:

In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import numpy as np
import pandas as pd
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.utils import CustomObjectScope
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score
from metrics import dice_loss, dice_coef, iou
from utils import get_filenames_sorted

In [ ]:
H = 512
W = 768

def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

def read_image(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.float32)
    return ori_x, x

def read_mask(path):
    x = cv2.imread(path, cv2.IMREAD_COLOR)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    ori_x = x
    x = x/255.0
    x = x.astype(np.float32)
    x = x[:,:,0:3] # Discard the last channel 
    return ori_x, x

def load_data(path):
    x = sorted(glob(os.path.join(path, "image", "*.jpg")))
    y = sorted(glob(os.path.join(path, "mask", "*.jpg")))
    return x, y

def save_results(ori_x, ori_y, y_pred, save_image_path, channel):
    line = np.ones((y_pred.shape[0], 10, 3)) * 255

    pred_image = np.zeros((y_pred.shape[0], y_pred.shape[1], 3))
    _y_pred = y_pred[:, :, channel]
    _ori_y = ori_y[:, :, channel]
    pred_image[:, :, 0] = ((_y_pred > 0.5) & (_ori_y <= 128)) * 255
    pred_image[:, :, 1] = ((_y_pred > 0.5) & (_ori_y  > 128)) * 255
    pred_image[:, :, 2] = ((_ori_y  > 128) & (_y_pred <= 0.5 )) * 255

    print(" saving result", save_image_path)
    cv2.imwrite(save_image_path, pred_image)

In [ ]:
data_dir = "new_data_augment"
od_channel, he_channel, ex_channel = 0, 1, 2


od_result_dir = "files_demo_bv"
he_result_dir = "files_demo_od"
ex_result_dir = "files_demo_ex"

create_dir(od_result_dir)
create_dir(he_result_dir)
create_dir(ex_result_dir)

model_dir = "files"

""" Load the model """
model_file_name = f"{model_dir}/model.h5"
print(model_file_name)
with CustomObjectScope({'iou': iou, 'dice_coef': dice_coef, 'dice_loss': dice_loss}):
    model = tf.keras.models.load_model(model_file_name)

""" Load the dataset """
# dataset_path = os.path.join(data_dir, "test")
test_x, test_y = get_filenames_sorted(data_dir + "/test/image/", data_dir + "/test/mask/" )

""" Make the prediction and calculate the metrics values """
SCORE_OD, SCORE_HE, SCORE_EX = [], [], []
for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
    """ Extracting name """
    name = x.rsplit("/", 1)[1].rsplit(".", 1)[0]
    print(name)

    """ Read the image and mask """
    ori_x, x = read_image(x)
    ori_y, y = read_mask(y)

    """ Prediction """
    y_pred = model.predict(np.expand_dims(x, axis=0))[0]
    y_pred = y_pred > 0.5
    y_pred = y_pred.astype(np.float32)

    print(np.max(ori_y), np.max(y))

    """ Saving the images """
    save_image_path_od = f"{od_result_dir}/{name}.png"
    save_results(ori_x, ori_y, y_pred, save_image_path_od, od_channel)

    save_image_path_he = f"{he_result_dir}/{name}.png"
    save_results(ori_x, ori_y, y_pred, save_image_path_he, he_channel)

    save_image_path_ex = f"{ex_result_dir}/{name}.png"
    save_results(ori_x, ori_y, y_pred, save_image_path_ex, ex_channel)


#     """ Calculate the OD metrics """
    ma_pred = y_pred[:, :, od_channel].flatten()
    ma_gt = y[:, :, od_channel].flatten()
    acc_value = accuracy_score(ma_gt > 0.5, ma_pred>0.5)
    f1_value = f1_score(ma_gt > 0.5, ma_pred>0.5, labels=[0, 1], average="binary")
    jac_value = jaccard_score(ma_gt > 0.5, ma_pred>0.5, labels=[0, 1], average="binary")
    recall_value = recall_score(ma_gt > 0.5, ma_pred>0.5, labels=[0, 1], average="binary")
    recall_computed = np.sum((ma_gt > 0.5) & (ma_pred > 0.5)) / np.sum(ma_gt > 0.5)
    precision_value = precision_score(ma_gt > 0.5, ma_pred>0.5, labels=[0, 1], average="binary")
    SCORE_OD.append([name, acc_value, f1_value, jac_value, recall_value, precision_value, recall_computed])

#     """ Calculate the HE metrics """
    bv_pred = y_pred[:, :, he_channel].flatten()
    bv_gt = y[:, :, he_channel].flatten()
    acc_value = accuracy_score(bv_gt > 0.5, bv_pred>0.5)
    f1_value = f1_score(bv_gt > 0.5, bv_pred>0.5, labels=[0, 1], average="binary")
    jac_value = jaccard_score(bv_gt > 0.5, bv_pred>0.5, labels=[0, 1], average="binary")
    recall_value = recall_score(bv_gt > 0.5, bv_pred>0.5, labels=[0, 1], average="binary")
    recall_computed = np.sum((bv_gt > 0.5) & (bv_pred > 0.5)) / np.sum(bv_gt > 0.5)
    precision_value = precision_score(bv_gt > 0.5, bv_pred>0.5, labels=[0, 1], average="binary")
    SCORE_HE.append([name, acc_value, f1_value, jac_value, recall_value, precision_value, recall_computed])

#     """ Calculate the EX metrics """
    he_pred = y_pred[:, :, ex_channel].flatten()
    he_gt = y[:, :, ex_channel].flatten()
    acc_value = accuracy_score(he_gt > 0.5, he_pred>0.5)
    f1_value = f1_score(he_gt > 0.5, he_pred>0.5, labels=[0, 1], average="binary")
    jac_value = jaccard_score(he_gt > 0.5, he_pred>0.5, labels=[0, 1], average="binary")
    recall_value = recall_score(he_gt > 0.5, he_pred>0.5, labels=[0, 1], average="binary")
    recall_computed = np.sum((he_gt > 0.5) & (he_pred > 0.5)) / np.sum(he_gt > 0.5)
    precision_value = precision_score(ma_gt > 0.5, ma_pred>0.5, labels=[0, 1], average="binary")
    SCORE_EX.append([name, acc_value, f1_value, jac_value, recall_value, precision_value, recall_computed])
    
print("\n")

for SCORE in [SCORE_OD, SCORE_HE, SCORE_EX]:
    if SCORE == SCORE_OD:
        print("****** OD Metrics ******")
        score = [s[1:] for s in SCORE]
        score = np.mean(score, axis=0)
        print(f"Accuracy: {score[0]:0.5f}")
        print(f"F1: {score[1]:0.5f} (dice score)")
        print(f"AUC: {score[1]:0.5f} (Auc score)")
        print(f"Jaccard: {score[2]:0.5f}")
        print(f"Recall: {score[3]:0.5f}")
        print(f"Precision: {score[4]:0.5f}")

    elif SCORE == SCORE_HE:
        print("****** HE Metrics ******")
        score = [s[1:] for s in SCORE]
        score = np.mean(score, axis=0)
        print(f"Accuracy: {score[0]:0.5f}")
        print(f"F1: {score[1]:0.5f} (dice score)")
        print(f"AUC: {score[1]:0.5f} (Auc score)")
        print(f"Jaccard: {score[2]:0.5f}")
        print(f"Recall: {score[3]:0.5f}")
        print(f"Precision: {score[4]:0.5f}")

    else:
        print("****** EX Metrics ******")
        score = [s[1:] for s in SCORE]
        score = np.mean(score, axis=0)
        print(f"Accuracy: {score[0]:0.5f}")
        print(f"F1: {score[1]:0.5f} (dice score)")
        print(f"AUC: {score[1]:0.5f} (Auc score)")
        print(f"Jaccard: {score[2]:0.5f}")
        print(f"Recall: {score[3]:0.5f}")
        print(f"Precision: {score[4]:0.5f}")

    # """ Saving """
    if SCORE == SCORE_OD:
        df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision", "Recall Computed"])
        df.to_csv(f"{od_result_dir}/score.csv")

    elif SCORE == SCORE_HE:
        df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision", "Recall Computed"])
        df.to_csv(f"{he_result_dir}/score.csv")
    else:
        df = pd.DataFrame(SCORE, columns=["Image", "Acc", "F1", "Jaccard", "Recall", "Precision", "Recall Computed"])
        df.to_csv(f"{ex_result_dir}/score.csv")
    print("\n")

files/model.h5


  0%|          | 0/27 [00:00<?, ?it/s]

IDRiD_55
255 1.0
 saving result files_demo_bv/IDRiD_55.png
 saving result files_demo_od/IDRiD_55.png
 saving result files_demo_ex/IDRiD_55.png


  4%|▎         | 1/27 [00:02<01:03,  2.44s/it]

IDRiD_56
255 1.0
 saving result files_demo_bv/IDRiD_56.png
 saving result files_demo_od/IDRiD_56.png
 saving result files_demo_ex/IDRiD_56.png


...

IDRiD_80
255 1.0
 saving result files_demo_bv/IDRiD_80.png
 saving result files_demo_od/IDRiD_80.png
 saving result files_demo_ex/IDRiD_80.png


 96%|█████████▋| 26/27 [00:49<00:01,  1.88s/it]

IDRiD_81
255 1.0
 saving result files_demo_bv/IDRiD_81.png
 saving result files_demo_od/IDRiD_81.png
 saving result files_demo_ex/IDRiD_81.png


100%|██████████| 27/27 [00:51<00:00,  1.90s/it]



****** OD Metrics ******
Accuracy: 0.98138
F1: 0.82181 (dice score)
AUC: 0.82181 (Auc score)
Jaccard: 0.69972
Recall: 0.79722
Precision: 0.86237


****** HE Metrics ******
Accuracy: 0.99792
F1: 0.93967 (dice score)
AUC: 0.93967 (Auc score)
Jaccard: 0.89354
Recall: 0.95615
Precision: 0.93615


****** EX Metrics ******
Accuracy: 0.99516
F1: 0.62460 (dice score)
AUC: 0.62460 (Auc score)
Jaccard: 0.47438
Recall: 0.58383
Precision: 0.86237


